The purpose of this is to compare the inputs from Ayako for imputation vs what we've got
for imputation.

In [1]:
import polars as pl

In [ ]:
expected = pl.read_parquet(
    "inputs/expected_imputation_2023_01"
)

actual = pl.read_parquet(
    "inputs/actual_imputed_2023_01.parquet",
)

In [5]:
str(expected.columns)

"['grid_id', 'date', 'aot_daily', 'co_daily', 'omi_no2', 'v_wind', 'u_wind', 'rainfall', 'temp', 'pressure', 'thermal_radiation', 'low_veg', 'high_veg', 'dewpoint_temp', 'NO2_tropos', 'NO2_missing', 'aod', 'aod_missing', 'CO', 'CO_missing', 'elevation', 'water', 'shurub', 'urban', 'forest', 'savannas', 'month', 'day_of_year', 'cos_day_of_year', 'monsoon', 'lon', 'lat', 'wind_degree', 'RH', 'aot_rolling', 'co_rolling', 'omi_no2_rolling', 'v_wind_rolling', 'u_wind_rolling', 'rainfall_rolling', 'temp_rolling', 'pressure_rolling', 'wind_degree_rolling', 'RH_rolling', 'thermal_radiation_rolling', 'dewpoint_temp_rolling', 'aot_daily_annual', 'co_daily_annual', 'omi_no2_annual', 'v_wind_annual', 'u_wind_annual', 'rainfall_annual', 'temp_annual', 'pressure_annual', 'thermal_radiation_annual', 'low_veg_annual', 'high_veg_annual', 'dewpoint_temp_annual', 'wind_degree_annual', 'RH_annual', 'aot_daily_allyears', 'co_daily_allyears', 'omi_no2_allyears', 'v_wind_allyears', 'u_wind_allyears', 'rainfa

In [6]:
str(actual.columns)

"['date', 'grid_id', 'temperature_2m', 'dewpoint_temperature_2m', 'u_component_of_wind_10m', 'v_component_of_wind_10m', 'total_precipitation_sum', 'surface_net_thermal_radiation_sum', 'surface_pressure', 'leaf_area_index_high_vegetation', 'leaf_area_index_low_vegetation', 'merra_aot', 'merra_co', 'Optical_Depth_047', 'Optical_Depth_055', 'omi_no2', 'CO_column_number_density', 'NO2_column_number_density', 'forest', 'shrub', 'savanna', 'urban', 'water', 'elevation']"

In [22]:
values_missing_temperature = actual.filter(
    pl.col("temperature_2m").is_null()
)[["date", "grid_id"]]

In [29]:
grid_id = 1278

In [30]:
expected[["date", "grid_id", "temp"]].filter(pl.col("grid_id") == grid_id)

date,grid_id,temp
datetime[ns],i64,f64
2023-01-01 00:00:00,1278,299.565007
2023-01-02 00:00:00,1278,299.511196
2023-01-03 00:00:00,1278,298.757951
2023-01-04 00:00:00,1278,299.21439
2023-01-05 00:00:00,1278,300.334467
…,…,…
2023-01-27 00:00:00,1278,298.238824
2023-01-28 00:00:00,1278,299.398999
2023-01-29 00:00:00,1278,298.807748


In [31]:
actual[["date", "grid_id", "temperature_2m"]].filter(pl.col("grid_id") == grid_id)

date,grid_id,temperature_2m
str,i64,f64
"""2023-01-01""",1278,null
"""2023-01-02""",1278,null
"""2023-01-03""",1278,null
"""2023-01-04""",1278,null
"""2023-01-05""",1278,null
…,…,…
"""2023-01-27""",1278,null
"""2023-01-28""",1278,null
"""2023-01-29""",1278,null


In [36]:
grid_id = 1607

In [37]:
expected[["date", "grid_id", "temp"]].filter(pl.col("grid_id") == grid_id)

date,grid_id,temp
datetime[ns],i64,f64
2023-01-01 00:00:00,1607,299.573755
2023-01-02 00:00:00,1607,299.53299
2023-01-03 00:00:00,1607,298.768977
2023-01-04 00:00:00,1607,299.237948
2023-01-05 00:00:00,1607,300.360488
…,…,…
2023-01-27 00:00:00,1607,298.238016
2023-01-28 00:00:00,1607,299.389701
2023-01-29 00:00:00,1607,298.84009


In [ ]:
actual[["date", "grid_id", "temperature_2m"]].filter(pl.col("grid_id") == grid_id)